### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import getpass
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.select import Select
import time
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from pprint import pprint

### Import Bik CSV w/ Features added 

In [2]:
df_bik = pd.read_csv('Bik_pt4.csv')
# df_bik['year'] = 'test'

df_bik['university_name'] = df_bik['First Author Affiliation']
df_bik['university_name'] = df_bik['university_name'].astype(str)

df_bik['university_name'] = [i.replace(' ', '') for i in df_bik['university_name']]

display(df_bik['university_name'])

0      Ludwig-Maximilians-UniversityofMunich
1             WashingtonUniversityinSt.Louis
2              UniversityofIllinoisatChicago
3                                        nan
4          UniversityofNebraskaMedicalCenter
                       ...                  
209            EcoleNormaleSupérieuredeParis
210                DelhiStateCancerInstitute
211                        HarvardUniversity
212            UniversityofWashingtonSeattle
213                          WuhanUniversity
Name: university_name, Length: 214, dtype: object

### Import University Rankings CSV 

In [4]:
df1 = pd.read_csv('university_rankings.csv')
df1 = df1.loc[df1['year'] == 2015]
df1 = df1.rename({'institution': 'university_name', }, axis=1)
display(df1)

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1200,735,Åbo Akademi University,Finland,9,367,567,218,765,679,645,646.0,731,44.32,2015
1201,881,École centrale de Lyon,France,45,367,255,218,856,841,812,991.0,570,44.15,2015
1202,548,École Centrale Paris,France,23,367,111,218,995,827,812,958.0,839,44.63,2015
1203,37,École normale supérieure - Paris,France,2,8,502,54,379,157,645,295.0,871,58.80,2015
1204,721,École normale supérieure de Cachan,France,32,121,323,218,895,776,428,826.0,871,44.33,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,337,York University,Canada,15,367,106,218,382,408,264,388.0,871,45.56,2015
2196,979,Yuan Ze University,Taiwan,20,367,567,218,923,984,812,946.0,871,44.05,2015
2197,191,Zhejiang University,China,4,367,309,218,71,290,368,265.0,106,47.66,2015
2198,984,Zhejiang University of Technology,China,79,367,567,218,858,991,812,958.0,672,44.04,2015


### Import University Demogrphics CSV

In [5]:
df2 = pd.read_csv('university_demographics.csv')
df2 = df2.loc[df2['year'] == 2015]
display(df2)


,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
1402,1,California Institute of Technology,United States of America,92.2,67.0,98.1,99.7,89.1,94.3,"2,243",6.9,27%,33 : 67,2015
1403,2,Harvard University,United States of America,92.9,67.6,98.6,98.9,44.0,93.3,"20,152",8.9,25%,NaN,2015
1404,3,University of Oxford,United Kingdom,88.6,90.7,97.7,95.5,72.9,93.2,"19,919",11.6,34%,46 : 54,2015
1405,4,Stanford University,United States of America,91.5,69.0,96.7,99.1,63.1,92.9,"15,596",7.8,22%,42 : 58,2015
1406,5,University of Cambridge,United Kingdom,89.7,87.8,95.6,95.2,51.1,92.0,"18,812",11.8,34%,46 : 54,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,351-400,Waseda University,Japan,25.5,43.1,16.3,55.5,32.6,-,"52,316",16.9,8%,35 : 65,2015
1799,351-400,Washington State University,United States of America,28.5,36.0,27.5,44.2,54.5,-,"24,550",18.3,7%,50 : 50,2015
1800,351-400,Western Sydney University,Australia,17.8,50.1,22.4,58.7,30.4,-,"30,704",32.2,9%,56 : 44,2015
1801,351-400,Wuhan University,China,34.1,33.1,17.4,48.5,85.9,-,"46,227",14.4,7%,NaN,2015


### Merge University Demographic + Ranking CSV. Then Clean Merged Datasets 

In [6]:
df3 = pd.merge(df1,df2,on='university_name',how='left')
df3['university_name'] = [i.replace('-', '') if '-' in i else i for i in df3['university_name']]
df3['university_name'] = [i.replace(' ', '') for i in df3['university_name']]
df3['university_name'] = [i.replace('The', '') if '-' in i else i for i in df3['university_name']]

df3['university_name'] = [i.strip() for i in df3['university_name']]

df3.columns
display(df3)


# df3.loc[df3['university_name'] == 'University of Washington  Seattle']

,world_rank_x,university_name,country_x,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations_x,...,international,research,citations_y,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year_y
0,735,ÅboAkademiUniversity,Finland,9,367,567,218,765,679,645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,881,ÉcolecentraledeLyon,France,45,367,255,218,856,841,812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,548,ÉcoleCentraleParis,France,23,367,111,218,995,827,812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,ÉcolenormalesupérieureParis,France,2,8,502,54,379,157,645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,721,ÉcolenormalesupérieuredeCachan,France,32,121,323,218,895,776,428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,337,YorkUniversity,Canada,15,367,106,218,382,408,264,...,59.7,35.3,62.9,34.1,-,"48,007",39.4,9%,59 : 41,2015.0
996,979,YuanZeUniversity,Taiwan,20,367,567,218,923,984,812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,191,ZhejiangUniversity,China,4,367,309,218,71,290,368,...,21.5,33.1,40.2,88.6,-,"47,508",15.9,5%,41 : 59,2015.0
998,984,ZhejiangUniversityofTechnology,China,79,367,567,218,858,991,812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Merge Bik CSV w/ University Merged dataframe

In [7]:
# df3 = pd.merge(df1,df2,on='university_name',how='left')
df_final = pd.merge(df_bik, df3,on='university_name',how='left')
display(df_final)


,Unnamed: 0,Authors,Title,Citation,DOI,Year,Month,0,1,2,...,international,research,citations_y,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year_y
0,0,"Inka Regine Weingaertner, Sarah Koutnik, Herma...",Chronic Morphine Treatment Attenuates Cell Gro...,"PLOS One, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053510,2013,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Jessica M. Esparza, Eileen OÕToole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,2013,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Sreedevi Avasarala, Fangfang Zhang, Guangliang...",Curcumin Modulates the Inflammatory Response a...,"PLOS ONE, February 2013 | Volume 8 | Issue 2 |...",10.1371/journal.pone.0057285,2013,2.0,NaN,NaN,1.0,...,53.4,34.1,55.1,40.7,-,"24,313",9.2,17%,53 : 47,2015.0
3,3,"Rounak Nassirpour, Pramod P. Mehta, Sangita M....",miR-221 Promotes Tumorigenesis in Human Triple...,"PLOS ONE, April 2013 | Volume 8 | Issue 4 | e6...",10.1371/journal.pone.0062170,2013,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Jianuo Liu, Peng Xu, Cory Collins, Han Liu, J...",HIV-1 Tat Protein Increases Microglial Outward...,"PLOS ONE, May 2013 | Volume 8 | Issue 5 | e649...",10.1371/journal.pone.0064904,2013,5.0,NaN,1.0,NaN,...,34.8,16.5,51.5,34.5,-,"2,857",2.6,8%,67 : 33,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,209,"Nathalie Delgehyr, Uta Wieland, He_lne Rangon...","Drosophila Mgr, a Prefoldin subunit cooperatin...","PNAS | April 10, 2012 | vol. 109 | no. 15 | 57...",10.1073/pnas.1108537109,2012,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,210,"Rajesh K. Grover, Julong Cheng, Yingjie Peng, ...",The costimulatory immunogen LPS induces the B-...,"PNAS _ April 17, 2012 _ vol. 109 _ no. 16, 603...",10.1073/pnas.1202214109,2012,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,211,"Kevin Y. Lee, Yuji Yamamoto, Jeremie Boucher, ...",Shox2 is a molecular determinant of depot-spec...,"PNAS | July 9, 2013 | vol. 110 | no. 28 | 1140...",10.1073/pnas.1310331110,2013,NaN,NaN,1.0,NaN,...,67.6,98.6,98.9,44.0,93.3,"20,152",8.9,25%,NaN,2015.0
212,212,"Shuyu Ren, Bryce G. Johnson, Yujiro Kida, Coli...",LRP-6 is a coreceptor for multiple fibrogenic ...,"PNAS | January22,2013 | vol.110 | no.4 | 1441",10.1073/pnas.1211179110,2013,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
selected_rows = df_final[~df_final['national_rank'].isnull()]
display(selected_rows) ### see how many values were merged 

,Unnamed: 0,Authors,Title,Citation,DOI,Year,Month,0,1,2,...,international,research,citations_y,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year_y
1,1,"Jessica M. Esparza, Eileen OÕToole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,2013,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"Sreedevi Avasarala, Fangfang Zhang, Guangliang...",Curcumin Modulates the Inflammatory Response a...,"PLOS ONE, February 2013 | Volume 8 | Issue 2 |...",10.1371/journal.pone.0057285,2013,2.0,NaN,NaN,1.0,...,53.4,34.1,55.1,40.7,-,"24,313",9.2,17%,53 : 47,2015.0
4,4,"Jianuo Liu, Peng Xu, Cory Collins, Han Liu, J...",HIV-1 Tat Protein Increases Microglial Outward...,"PLOS ONE, May 2013 | Volume 8 | Issue 5 | e649...",10.1371/journal.pone.0064904,2013,5.0,NaN,1.0,NaN,...,34.8,16.5,51.5,34.5,-,"2,857",2.6,8%,67 : 33,2015.0
6,6,"Elizabeth Ngoc Hoa Tran, Matthew Thomas Doyle,...",LPS Unmasking of Shigella flexneri Reveals Pre...,"PLOS ONE, July 2013 | Volume 8 | Issue 7 | e70...",10.1371/journal.pone.0070508,2013,7.0,1.0,NaN,NaN,...,79.3,40.4,61.1,65.3,48.2,"20,771",30.1,26%,48 : 52,2015.0
8,8,"Jia Ma, Qing Zhang, Sulian Chen, Binbin Fang, ...",Mitochondrial Dysfunction Promotes Breast Canc...,"PLOS ONE, July 2013 | Volume 8 | Issue 7 | e69...",10.1371/journal.pone.0069485,2013,7.0,NaN,NaN,NaN,...,68.3,79.4,95.3,-,84.4,"25,055",5.9,28%,NaN,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,"MICHIYO YAMAMURA, KAZUMA NOGUCHI, YOSHIRO NAKA...",Functional analysis of Zyxin in cell migration...,"INTERNATIONAL JOURNAL OF ONCOLOGY 42: 873-880,...",10.3892/ijo.2013.1761,2013,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,203,"Steffen Zender, Irina Nickeleit, Torsten Wuest...",A Critical Role for Notch Signaling in the For...,"Cancer Cell 23, 784Ð795, June 10, 2013",10.1016/j.ccr.2013.04.019,2013,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,211,"Kevin Y. Lee, Yuji Yamamoto, Jeremie Boucher, ...",Shox2 is a molecular determinant of depot-spec...,"PNAS | July 9, 2013 | vol. 110 | no. 28 | 1140...",10.1073/pnas.1310331110,2013,NaN,NaN,1.0,NaN,...,67.6,98.6,98.9,44.0,93.3,"20,152",8.9,25%,NaN,2015.0
212,212,"Shuyu Ren, Bryce G. Johnson, Yujiro Kida, Coli...",LRP-6 is a coreceptor for multiple fibrogenic ...,"PNAS | January22,2013 | vol.110 | no.4 | 1441",10.1073/pnas.1211179110,2013,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Import Unemployment Text Data + Clean Text File 

In [9]:

file = open('countydata.txt', encoding='utf8')

data = {'county': [],'labor_force':[],'employed':[],'unemployed':[],'rate':[]}

full_list = list()

for row in file:
#     print(row)
    row = row.strip().split("\t")
    
    
    row = str(row)

    row = row.split(' ')
    row = [s for s in row if s]
#     print(len(row))


    if len(row) == 11: 
        data['county'].append(row[3]+row[4])
        data['labor_force'].append(row[7])
        data['employed'].append(row[8])
        data['unemployed'].append(row[9])
        data['rate'].append(row[10])
        
    elif len(row) == 12:
        
        data['county'].append(row[3]+row[4])
        data['labor_force'].append(row[8])
        data['employed'].append(row[9])
        data['unemployed'].append(row[10])
        data['rate'].append(row[11])
        
    elif len(row) == 13:
        
        data['county'].append(row[3]+row[4])
        data['labor_force'].append(row[9])
        data['employed'].append(row[10])
        data['unemployed'].append(row[11])
        data['rate'].append(row[12])
        
    elif len(row) == 14:
        
        data['county'].append(row[3]+row[4]+row[5])
        data['labor_force'].append(row[10])
        data['employed'].append(row[11])
        data['unemployed'].append(row[12])
        data['rate'].append(row[13])
        
    else: 
        print(row)
        break

unemployment = pd.DataFrame.from_dict(data)   

display(unemployment)


,county,labor_force,employed,unemployed,rate
0,"AutaugaCounty,","25,838","24,576","1,262",4.9']
1,"BaldwinCounty,","96,763","91,338","5,425",5.6']
2,"BarbourCounty,","8,587","7,982",605,7.0']
3,"BibbCounty,","8,640","8,067",573,6.6']
4,"BlountCounty,","24,661","23,653","1,008",4.1']
...,...,...,...,...,...
3214,VegaBaja,N.A.,N.A.,N.A.,N.A.']
3215,"ViequesMunicipio,",N.A.,N.A.,N.A.,N.A.']
3216,"VillalbaMunicipio,",N.A.,N.A.,N.A.,N.A.']
3217,"YabucoaMunicipio,",N.A.,N.A.,N.A.,N.A.']


In [10]:
data['rate'] = [i[:3] for i in data['rate']]
data['county'] = [i.replace(',', '') for i in data['county']]

# print(data)

unemployment = pd.DataFrame.from_dict(data)   

unemployment['county'].unique()

#lower case and remove space so that we can do an exact match on county later 

unemployment['county'] = [i.lower() for i in unemployment['county']] 
unemployment['county'] = [i.replace('county','') for i in unemployment['county']] 

unemployment = unemployment.drop_duplicates(subset = ["county"]) 


display(unemployment.head(30))


,county,labor_force,employed,unemployed,rate
0,autauga,"25,838","24,576","1,262",4.9
1,baldwin,"96,763","91,338","5,425",5.6
2,barbour,"8,587","7,982",605,7.0
3,bibb,"8,640","8,067",573,6.6
4,blount,"24,661","23,653","1,008",4.1
5,bullock,"4,818","4,553",265,5.5
6,butler,"9,056","8,255",801,8.8
7,calhoun,"46,240","42,980","3,260",7.1
8,chambers,"15,865","14,787","1,078",6.8
9,cherokee,"11,513","10,983",530,4.6


### Import U.S. Cities & Counties CSV to merge unemployment data by county 

In [11]:
citiesdf = pd.read_csv('uscities.csv')

citiesdf = citiesdf.drop_duplicates(subset = ["city"])

# display(citiesdf)
citiesdf['county_name_test'] = citiesdf['county_name']

citiesdf['county_name_test'] = [i.lower() for i in citiesdf['county_name_test']]
citiesdf['county_name_test'] = [i.replace(' ','') for i in citiesdf['county_name_test']]

citiesdf['city'] = [i.lower() for i in citiesdf['city']]
citiesdf['city'] = [i.replace(' ','') for i in citiesdf['city']]


# display(citiesdf['county_name_test'])

display(citiesdf)

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,county_name_test
0,newyork,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016,newyork
1,losangeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491,losangeles
2,chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494,cook
3,miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149,miami-dade
4,dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440,dallas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28332,portagecreek,Portage Creek,AK,Alaska,2070,Dillingham,58.9051,-157.6695,3,0,polygon,False,False,America/Anchorage,3,99576,1840023660,dillingham
28333,gross,Gross,NE,Nebraska,31015,Boyd,42.9461,-98.5697,2,6,polygon,False,True,America/Chicago,3,68719,1840011032,boyd
28334,lotsee,Lotsee,OK,Oklahoma,40143,Tulsa,36.1334,-96.2091,2,39,polygon,False,True,America/Chicago,3,74063,1840021674,tulsa
28335,theranch,The Ranch,MN,Minnesota,27087,Mahnomen,47.3198,-95.6952,2,2,polygon,False,True,America/Chicago,3,56557,1840039629,mahnomen


In [13]:
df_final['First Author Affiliation'] = df_final['First Author Affiliation'].astype(str) #converts column to string 

### Find the city of each university name via google search

In [19]:
### this proces will take 20-60 minute 
from googlesearch import search 
full_address = []

for i in range(len(df_final['First Author Affiliation'])):
    # for each university name - serach university name + mailing address 
    query = df_final['First Author Affiliation'][i] + " mailing address" 
    for j in search(query, tld="com", num=1, stop=1, pause=2):
        browser_driver = Service('chromedriver.exe') #this has to path to chromedriver.exe 

#         browser_driver = Service('C:/Users/schoo/Downloads/chromedriver_win32/chromedriver.exe')

        page_to_scrape = webdriver.Chrome(service=browser_driver)
        page_to_scrape.get("https://www.google.com/")
        email = page_to_scrape.find_element(By.NAME, '''q''')

        email.send_keys(query,Keys.ENTER)

# time.sleep(0.2)

    get_url = page_to_scrape.current_url
    # print(get_url)
        
    try:    
        response = requests.get(get_url)
        soup = BeautifulSoup(response.text, 'html.parser') #shows format 
        table_extract = soup.find('div',class_='BNeawe iBp4i AP7Wnd').text #extract the mailing address 
        print(table_extract)
        
        full_address.append(table_extract) #append to list 
        
    except:
        full_address.append('')
        print('None Found')
        continue


Geschwister-Scholl-Platz 1, 80539 München, Germany
1 Brookings Dr, St. Louis, MO 63130
1200 W Harrison St, Chicago, IL 60607
None Found
42nd and, Emile St, Omaha, NE 68198
None Found
Adelaide SA 5005, Australia
Piazza Umberto I, 1, 70121 Bari BA, Italy
New York, NY 10027
Kalman Ya'akov Man St, Jerusalem, Israel
None Found
None Found
Glasgow G12 8QQ, UK
None Found
77 Yongbong-ro, Buk-gu, Kwangju, South Korea
201 Presidents' Cir, Salt Lake City, UT 84112
Šlajmerjeva ulica 6, 1000 Ljubljana, Slovenia
Wuzhong District, Suzhou, China, 215000
80 Av. Augustin Fliche, 34090 Montpellier, France
None Found
67 Asahimachi, Kurume, Fukuoka 830-0011, Japan
Ludwigstraße 23, 35390 Gießen, Germany
China, Chong Qing Shi, Nan An Qu, 南坪东路7号重庆迎龙化工有限公司B楼4F
None Found
Cidade Universitária Zeferino Vaz - Barão Geraldo, Campinas - SP, 13083-970, Brazil
None Found
Blacksburg, VA 24061
Madison, WI
25 Shattuck St, Boston, MA 02115
Aruna Asaf Ali Marg, Jawaharlal Nehru University, New Delhi, Delhi 110067, India
No

In [22]:
saving = full_address
print(len(full_address))

214


In [26]:
cities = []
for i in full_address: 
    comma_count = i.count(',') 
#     print(comma_count)
    
    
    if comma_count ==2: 
        split_string= i.split(',')
        split_string[1] = split_string[1].strip()
        
        cities.append(split_string[1])
    
    elif comma_count ==1: 
        split_string= i.split(',')
        cities.append(split_string[0])
        
    else:
        cities.append('')
        

cities= [i.lower() for i in cities]
cities = [i.replace(' ','') for i in cities]


print(cities)
# bik_cities_df = pd.DataFrame(univ_list, columns = ['university_name'])
df_final['cities'] = cities
# bik_cities_df


['80539münchen', 'st.louis', 'chicago', '', '', '', 'adelaidesa5005', '', 'newyork', 'jerusalem', '', '', 'glasgowg128qq', '', '', 'saltlakecity', '1000ljubljana', '', '34090montpellier', '', '', '35390gießen', '', '', '', '', 'blacksburg', 'madison', 'boston', '', '', 'atlanta', 'iowacity', '', 'houston', '75236uppsala', '', 'boston', '', '', 'columbus', '', 'newark', '', '75236uppsala', '8collegerd', '', '', '', '', '', '', 'batonrouge', 'batonrouge', '', 'newyork', 'charlotte', 'pittsburgh', 'collegepark', 'batonrouge', 'saltlakecity', 'losangeles', 'annarbor', '', '', '', '', 'cleveland', 'durham', '', 'boston', '', '91054erlangen', '', '', 'medford', 'medford', 'minneapolis', '', '', 'columbus', 'corvallis', '', '', 'pokfulam', '', '', 'corvallis', '', '', 'penang', '', 'district6', '', '', '', '', '', 'c1053caba', '', '', '', '', '', '', '', 'losangeles', '', '', '', '', '', '', '', '', '', '', 'syracuse', 'hobarttas7005', '', '', '', '', '', '', '', '', 'belfastbt71nn', '', '', 

### Merge ALL data Together 

In [30]:
merged = pd.merge(df_final, citiesdf, left_on='cities', right_on='city', how='left') #merge cities founded from google search to cities/counties database. This step is needed to find the county for each university 

merged2 = pd.merge(merged, unemployment, left_on='county_name_test', right_on='county', how='left') #merge county names founded w/ county labor data 

# display(merged2)

merged3 = pd.concat([df_final, merged2], axis=1)
merged3
# merged3 = pd.merge(df_final, merged2,on='university_name',how='left')



,Unnamed: 0,Authors,Title,Citation,DOI,Year,Month,0,1,2,...,timezone,ranking,zips,id,county_name_test,county,labor_force,employed,unemployed,rate
0,0,"Inka Regine Weingaertner, Sarah Koutnik, Herma...",Chronic Morphine Treatment Attenuates Cell Gro...,"PLOS One, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053510,2013,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Jessica M. Esparza, Eileen OÕToole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,2013,1.0,NaN,1.0,NaN,...,America/Chicago,1.0,63110 63111 63112 63113 63115 63116 63118 6312...,1.840002e+09,st.louis,st.louis,"101,110","93,824","7,286",7.2
2,2,"Sreedevi Avasarala, Fangfang Zhang, Guangliang...",Curcumin Modulates the Inflammatory Response a...,"PLOS ONE, February 2013 | Volume 8 | Issue 2 |...",10.1371/journal.pone.0057285,2013,2.0,NaN,NaN,1.0,...,America/Chicago,1.0,60018 60649 60641 60640 60643 60642 60645 6064...,1.840000e+09,cook,cook,"8,102","7,718",384,4.7
3,3,"Rounak Nassirpour, Pramod P. Mehta, Sangita M....",miR-221 Promotes Tumorigenesis in Human Triple...,"PLOS ONE, April 2013 | Volume 8 | Issue 4 | e6...",10.1371/journal.pone.0062170,2013,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"Jianuo Liu, Peng Xu, Cory Collins, Han Liu, J...",HIV-1 Tat Protein Increases Microglial Outward...,"PLOS ONE, May 2013 | Volume 8 | Issue 5 | e649...",10.1371/journal.pone.0064904,2013,5.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,209,"Nathalie Delgehyr, Uta Wieland, He_lne Rangon...","Drosophila Mgr, a Prefoldin subunit cooperatin...","PNAS | April 10, 2012 | vol. 109 | no. 15 | 57...",10.1073/pnas.1108537109,2012,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,210,"Rajesh K. Grover, Julong Cheng, Yingjie Peng, ...",The costimulatory immunogen LPS induces the B-...,"PNAS _ April 17, 2012 _ vol. 109 _ no. 16, 603...",10.1073/pnas.1202214109,2012,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,211,"Kevin Y. Lee, Yuji Yamamoto, Jeremie Boucher, ...",Shox2 is a molecular determinant of depot-spec...,"PNAS | July 9, 2013 | vol. 110 | no. 28 | 1140...",10.1073/pnas.1310331110,2013,NaN,NaN,1.0,NaN,...,America/New_York,2.0,02142 02140 02141 02139 02138 02238 02477,1.840000e+09,middlesex,middlesex,"92,221","86,211","6,010",6.5
212,212,"Shuyu Ren, Bryce G. Johnson, Yujiro Kida, Coli...",LRP-6 is a coreceptor for multiple fibrogenic ...,"PNAS | January22,2013 | vol.110 | no.4 | 1441",10.1073/pnas.1211179110,2013,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
selected_rows = merged3[~merged3['county'].isnull()]
display(selected_rows) ### see how many are merged 

,Unnamed: 0,Authors,Title,Citation,DOI,Year,Month,0,1,2,...,timezone,ranking,zips,id,county_name_test,county,labor_force,employed,unemployed,rate
1,1,"Jessica M. Esparza, Eileen OÕToole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,2013,1.0,NaN,1.0,NaN,...,America/Chicago,1.0,63110 63111 63112 63113 63115 63116 63118 6312...,1.840002e+09,st.louis,st.louis,"101,110","93,824","7,286",7.2
2,2,"Sreedevi Avasarala, Fangfang Zhang, Guangliang...",Curcumin Modulates the Inflammatory Response a...,"PLOS ONE, February 2013 | Volume 8 | Issue 2 |...",10.1371/journal.pone.0057285,2013,2.0,NaN,NaN,1.0,...,America/Chicago,1.0,60018 60649 60641 60640 60643 60642 60645 6064...,1.840000e+09,cook,cook,"8,102","7,718",384,4.7
8,8,"Jia Ma, Qing Zhang, Sulian Chen, Binbin Fang, ...",Mitochondrial Dysfunction Promotes Breast Canc...,"PLOS ONE, July 2013 | Volume 8 | Issue 7 | e69...",10.1371/journal.pone.0069485,2013,7.0,NaN,NaN,NaN,...,America/New_York,1.0,11229 11226 11225 11224 11222 11221 11220 1138...,1.840034e+09,newyork,newyork,"859,618","777,583","82,035",9.5
9,9,"Yael Friedman-Levi, Michal Mizrahi, Kati Frid,...","PrPST, a Soluble, Protease Resistant and Trunc...","PLOS ONE, July 2013 | Volume 8 | Issue 7 | e69...",10.1371/journal.pone.0069583,2013,7.0,NaN,NaN,NaN,...,America/New_York,3.0,43747,1.840013e+09,monroe,monroe,"7,474","6,864",610,8.2
15,15,"Trent Fowler, Payel Ghatak, David H. Price, Ro...",Regulation of MYC Expression and Differential ...,"PLOS One, January 2014 | Volume 9 | Issue 1 | ...",10.1371/journal.pone.0087003,2014,1.0,NaN,NaN,1.0,...,America/Denver,1.0,84116 84115 84113 84112 84111 84144 84128 8418...,1.840021e+09,saltlake,saltlake,"642,357","609,766","32,591",5.1
26,26,"Xiaomei Ge, Yafei Zhang, Sungwon Park, Xiaofei...",Stac3 Inhibits Myoblast Differentiation into M...,"PLOS One, April 2014 | Volume 9 | Issue 4 | e9...",10.1371/journal.pone.0095926,2014,4.0,NaN,1.0,NaN,...,America/New_York,3.0,24060 24061 24062 24063 24111,1.840006e+09,montgomery,montgomery,"107,103","98,735","8,368",7.8
27,27,"Xudong Shi, Daniel DiRenzo, Lian-Wang Guo, Sar...",TGF-b/Smad3 Stimulates Stem Cell/Developmental...,"PLOS One, April 2014 | Volume 9 | Issue 4 | e9...",10.1371/journal.pone.0093995,2014,4.0,NaN,1.0,NaN,...,America/Chicago,2.0,53706 53704 53705 53703 53726 53792 53719 5371...,1.840003e+09,dane,dane,"319,433","304,154","15,279",4.8
28,28,"Yushen Jiang, Tang Jiang, Juan Ouyang, Qingson...",Cell Atavistic Transition: Paired Box 2 Re-Exp...,"PLOS One, April 2014 | Volume 9 | Issue 4 | e9...",10.1371/journal.pone.0093563,2014,4.0,NaN,1.0,NaN,...,America/New_York,1.0,02120 02121 02122 02124 02125 02126 02127 0212...,1.840000e+09,suffolk,suffolk,"446,558","403,388","43,170",9.7
31,31,"Yang Liu, Yuanyuan Liu, Chao Sun, Lu Gan, Luwe...",Carbon Ion Radiation Inhibits Glioma and Endot...,"PLOS ONE, June 2014 | Volume 9 | Issue 6 | e98448",10.1371/journal.pone.0098448,2014,6.0,NaN,NaN,1.0,...,America/New_York,1.0,30334 30331 30332 30309 30308 30305 30307 3030...,1.840014e+09,fulton,fulton,"4,924","4,671",253,5.1
32,32,"Manasa K. Nayak, Ayusman Dash, Nitesh Singh, D...",Aspirin Delimits Platelet Life Span by Proteas...,"PLOS ONE, August 2014 | Volume 9 | Issue 8 | e...",10.1371/journal.pone.0105049,2014,8.0,NaN,NaN,NaN,...,America/Chicago,2.0,52245 52246 52240 52242 52243 52244,1.840001e+09,johnson,johnson,"10,120","9,482",638,6.3


### Drop Irrelevant Columns and Export to CSV 

In [1]:
merged3 = merged3.drop(['id', 'county_name_test','city','cities'], axis=1)
merged3.to_csv('Bik_pt5.csv',index=False)



NameError: name 'merged3' is not defined